In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/onlyragtruth/my_model_final-llama-onllyragtruth/adapter_model.safetensors
/kaggle/input/onlyragtruth/my_model_final-llama-onllyragtruth/training_args.bin
/kaggle/input/onlyragtruth/my_model_final-llama-onllyragtruth/adapter_config.json
/kaggle/input/onlyragtruth/my_model_final-llama-onllyragtruth/README.md
/kaggle/input/last-corpus/mushroom.zh-tst.v1.jsonl
/kaggle/input/last-corpus/zh-last-query.json
/kaggle/input/last-corpus/zh-now-query.json
/kaggle/input/last-corpus/mushroom.en-tst.v1.jsonl
/kaggle/input/last-corpus/en-now-query.json
/kaggle/input/last-corpus/mushroom.zh-val.v2.jsonl
/kaggle/input/last-corpus/en-last-query.json
/kaggle/input/last-corpus/mushroom.en-val.v2.jsonl
/kaggle/input/text-en/mushroom.en-val.v1.jsonl
/kaggle/input/review2-0/en-now-query.json
/kaggle/input/modellora/my_model_halu-rag--llama 3b/adapter_model.safetensors
/kaggle/input/modellora/my_model_halu-rag--llama 3b/training_args.bin
/kaggle/input/modellora/my_model_halu-rag--llama 3b/adapter

In [ ]:
!pip install peft
!pip install accelerate
!pip install -U bitsandbytes

In [ ]:
import re
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, XLNetTokenizerFast,LlamaModel,LlamaTokenizerFast,PreTrainedTokenizerFast,BitsAndBytesConfig
import huggingface_hub 
from transformers import LlamaForCausalLM
from transformers import Trainer, TrainingArguments
#from sklearn.model_selection import train_test_split
import json
from huggingface_hub import login
import torch
from peft import LoraConfig,get_peft_model
import wandb

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
lora_config = LoraConfig(
    r = 16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="TOKEN_CLS"     
)




tokenizer = LlamaTokenizerFast.from_pretrained('meta-llama/Llama-3.2-3B')
tokenizer.pad_token = tokenizer.decode(128004)

In [4]:
import json
import os


import joblib
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel



with open('/kaggle/input/last-corpus/mushroom.zh-val.v2.jsonl', 'r', encoding='utf-8') as file:
    data = []
    for line in file:
        data.append(json.loads(line.strip()))
with open('/kaggle/input/last-corpus/zh-last-query.json', "r", encoding="utf-8") as json_file:
    queries = json.load(json_file)
for i in range(len(data)):
    data[i]['context'] = queries.get(data[i]['model_input'])
train = Dataset.from_list(data)
train =train.remove_columns(['lang','model_id','model_output_logits','model_output_tokens'])


end_token = tokenizer.decode(128001)
start_token = tokenizer.decode(128000)


In [5]:
dataset = train.map(preprocess,batched=False)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [6]:
id2label = {
    0: 'B-HAL',  
    1: 'I-HAL',  
    2: 'E-HAL',  
    3: 'O'       
}
label2id = {
    'B-HAL': 0,
    'I-HAL': 1,
    'E-HAL': 2,
    'O': 3
}
from transformers import LlamaForTokenClassification,AutoModelForTokenClassification
from peft import PeftModel

model = LlamaForTokenClassification.from_pretrained('meta-llama/Llama-3.2-3B', num_labels=4, id2label=id2label, label2id=label2id,output_hidden_states=True)

model = PeftModel.from_pretrained(model,"/kaggle/input/modellora/my_model-ragl-llama 3b")



config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def preprocess(tmp):


    input_text = f"{tmp.get('model_input')}{end_token}{start_token}{tmp.get('context')}{end_token}"
    text_pair = f"{tmp.get('model_output_text')}"
    inputs = tokenizer(text=input_text, text_pair=text_pair, return_tensors='pt',
                       return_offsets_mapping=True).to('cuda')

    inputs['input_ids']=inputs['input_ids']
    inputs['attention_mask']=inputs['attention_mask']
    tmp = inputs.encodings[0].offsets
    return tmp,inputs

In [12]:
with open('/kaggle/input/last-corpus/mushroom.en-tst.v1.jsonl', 'r', encoding='utf-8') as file:
    data = []
    for line in file:
        data.append(json.loads(line.strip()))
with open('/kaggle/input/review2-0/en-now-query.json', "r", encoding="utf-8") as json_file:
    queries = json.load(json_file)
for i in range(len(data)):
    data[i]['context'] = queries.get(data[i]['model_input'])
test = Dataset.from_list(data)
test =test.remove_columns(['id','lang','model_id','model_output_logits','model_output_tokens'])

In [15]:
import torch.nn.functional as F
model.to('cuda')
def softmax(logits):
    
    probabilities = F.softmax(logits,dim=0)  
    return probabilities
wait = [] 
with torch.no_grad():
    for i in range(len(test)):
        mapp,ret = preprocess(test[i])
        ret.pop('offset_mapping')
        mapp = mapp
        
        ret= model(**ret).logits
        predicted_token_class_ids = ret.argmax(-1)
        prob= ret[0]
        j =len(ret)-1
        
        predicted_tokens_classes = [id2label[t.item()] for t in predicted_token_class_ids[0]]
        labels = predicted_token_class_ids[0].tolist()
        i = len(mapp)-1
        ret = []
        tmp =[]
        
        while i>0:
            if mapp[i][0]==0 and mapp[i][1]==0:
                break
            if labels[i]!=3:
                if len(tmp)==0:
                    tmp.append(mapp[i][0])
                    tmp.append(mapp[i][1])
                    
                else:
                    tmp[0] = mapp[i][0]
                    
                    if labels[i]==0:
                        
                        probabilities = softmax(prob[j])
                        tmp.append(probabilities[probabilities.argmax(-1)])
                        ret.append(tmp)
                        tmp = []
            else:
                if len(tmp)>0:
                    
                    probabilities = softmax(prob[j-1])
                    tmp.append(probabilities[probabilities.argmax(-1)])
                    ret.append(tmp)
                    tmp = []
            i-=1
            j-=1
        wait.append(ret)    
        


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [16]:
with open('/kaggle/input/last-corpus/mushroom.en-tst.v1.jsonl', 'r', encoding='utf-8') as file:
    data = []
    for line in file:
        data.append(json.loads(line.strip()))